## Preparación de Datos
1. Split de datos:
   - Train/Test/Validation
   - Mantener estratificación si es necesario
2. Procesamiento de variables (fit solo en train):
   - Codificación de variables categóricas
   - Normalización/Estandarización de variables numéricas
3. Balanceo de clases (solo en train si es necesario):
   - Oversampling
   - Undersampling
   - Técnicas híbridas

### Librerías

In [49]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import re

### Datos

In [50]:
df1 = pd.read_csv(r'C:\Users\nuria\OneDrive\Escritorio\ML_laptops\notebooks\DataFrame_laptops_limpio_1')
df1.head()

,Unnamed: 0,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,...,velocidad_cpu_ghz,marca_gpu,modelo_gpu,OpSys_general,2 in 1 Convertible,Gaming,Netbook,Notebook,Ultrabook,Workstation
0,0,1223,Dell,Inspiron 5567,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,3,256GB SSD,...,2.5,AMD,Radeon R7 M445,Windows,0,0,0,1,0,0
1,1,78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,3,2TB HDD,...,2.5,Intel,HD Graphics 620,Sin OS,0,0,0,1,0,0
2,2,1267,Dell,XPS 13,2 in 1 Convertible,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i5 7Y54 1.2GHz,3,256GB SSD,...,1.2,Intel,HD Graphics 615,Windows,1,0,0,0,0,0
3,3,161,Dell,Inspiron 5579,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,3,256GB SSD,...,1.8,Intel,UHD Graphics 620,Windows,1,0,0,0,0,0
4,4,922,LG,Gram 14Z970,Ultrabook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,3,512GB SSD,...,2.7,Intel,HD Graphics 620,Windows,0,0,0,0,1,0


In [51]:
df1.isnull().sum()

Unnamed: 0            0
laptop_ID             0
Company               0
Product               0
TypeName              0
Inches                0
ScreenResolution      0
Cpu                   0
Ram                   0
Memory                0
Gpu                   0
OpSys                 0
Weight                0
Price_euros           0
Resolución            0
tipo_pantalla         0
memoria               0
tipo_memoria          0
tipo_cpu              0
Marca_cpu             0
Serie_cpu             0
Modelo_cpu            0
velocidad_cpu_ghz     0
marca_gpu             0
modelo_gpu            0
OpSys_general         0
2 in 1 Convertible    0
Gaming                0
Netbook               0
Notebook              0
Ultrabook             0
Workstation           0
dtype: int64

In [52]:
# Convertir a numérico usando el método de codificación
df1['OpSys'] = df1['OpSys'].astype('category').cat.codes
df1['velocidad_cpu_ghz'] = df1['velocidad_cpu_ghz'].astype('category').cat.codes

In [53]:
# Convertir todas las columnas a numéricas, forzando a NaN donde no se pueda
# df_numeric = df1.apply(pd.to_numeric, errors='coerce')

## Procesado para ML

In [54]:
df1.head(7)

,Unnamed: 0,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,...,velocidad_cpu_ghz,marca_gpu,modelo_gpu,OpSys_general,2 in 1 Convertible,Gaming,Netbook,Notebook,Ultrabook,Workstation
0,0,1223,Dell,Inspiron 5567,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,3,256GB SSD,...,16,AMD,Radeon R7 M445,Windows,0,0,0,1,0,0
1,1,78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,3,2TB HDD,...,16,Intel,HD Graphics 620,Sin OS,0,0,0,1,0,0
2,2,1267,Dell,XPS 13,2 in 1 Convertible,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i5 7Y54 1.2GHz,3,256GB SSD,...,3,Intel,HD Graphics 615,Windows,1,0,0,0,0,0
3,3,161,Dell,Inspiron 5579,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,3,256GB SSD,...,8,Intel,UHD Graphics 620,Windows,1,0,0,0,0,0
4,4,922,LG,Gram 14Z970,Ultrabook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,3,512GB SSD,...,18,Intel,HD Graphics 620,Windows,0,0,0,0,1,0
5,5,1040,HP,ProBook 650,Notebook,14.0,1366x768,Intel Core i5 7200U 2.5GHz,1,500GB HDD,...,16,Intel,HD Graphics 620,Windows,0,0,0,1,0,0
6,6,1039,HP,Elitebook 820,Ultrabook,12.5,1366x768,Intel Core i5 7200U 2.5GHz,1,256GB SSD,...,16,Intel,HD Graphics 620,Windows,0,0,0,0,1,0


### 1. Definir X e y

In [73]:
X = df1.drop(['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Memory', 'marca_gpu','modelo_gpu','OpSys_general', 'Gpu', 'Price_euros','Resolución','tipo_pantalla','memoria', 'Marca_cpu','Serie_cpu','Modelo_cpu', 'Weight',	'tipo_memoria',	'tipo_cpu'], axis=1)
y = df1['Price_euros'].copy()
X.shape

(912, 12)

In [74]:
y.shape

(912,)

### 2. Dividir X_train, X_test, y_train, y_test

In [75]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [76]:
X_train

,Unnamed: 0,laptop_ID,Inches,Ram,OpSys,velocidad_cpu_ghz,2 in 1 Convertible,Gaming,Netbook,Notebook,Ultrabook,Workstation
25,25,439,14.0,1,5,18,0,0,0,1,0,0
84,84,979,15.6,1,2,11,0,0,0,1,0,0
10,10,337,15.6,3,5,16,0,0,0,1,0,0
342,342,139,15.6,1,4,2,0,0,0,1,0,0
890,890,517,13.3,3,5,16,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
106,106,474,15.6,3,5,7,0,0,0,1,0,0
270,270,478,15.6,3,5,16,0,0,0,1,0,0
860,860,952,14.0,3,5,16,0,0,0,0,1,0
435,435,370,15.6,3,4,11,0,0,0,1,0,0


### 3. Asignar el modelo (vacío) a una variable

In [77]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()

regressor.fit(X_train, y_train) # Entrenar

LinearRegression()

### Sacar métricas, valorar el modelo
RMSE para la competición

In [78]:
from sklearn.metrics import root_mean_squared_error

predictions = regressor.predict(X_test)

rmse = root_mean_squared_error(y_test, predictions)

In [79]:
print(rmse)

421.34656906732846


## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [67]:
X_pred = pd.read_csv(r"C:\Users\nuria\OneDrive\Escritorio\ML_laptops\data\raw_data\test.csv")
X_pred.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,539,Asus,Zenbook UX510UW-FI095T,Notebook,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 960M,Windows 10,2kg
1,327,Asus,ZenBook UX410UA-GV183T,Notebook,14.0,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,2kg
2,563,Mediacom,SmartBook 130,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Atom x5-Z8350 1.44GHz,4GB,32GB Flash Storage,Intel HD Graphics,Windows 10,1.35kg
3,13,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.8GHz,16GB,256GB SSD,AMD Radeon Pro 555,macOS,1.83kg
4,935,HP,EliteBook 850,Ultrabook,15.6,Full HD 1920x1080,Intel Core i7 6500U 2.5GHz,8GB,256GB SSD,AMD Radeon R7 M365X,Windows 10,1.84kg


In [68]:
X_pred.shape

(391, 12)

In [84]:
X_pred = X_pred[['laptop_ID', 'Inches']]
X_pred

,laptop_ID,Inches
0,539,15.6
1,327,14.0
2,563,13.3
3,13,15.4
4,935,15.6
...,...,...
386,742,13.3
387,660,13.3
388,983,15.6
389,1137,14.0


In [85]:
print(X_pred.isnull().sum())

laptop_ID    0
Inches       0
dtype: int64


In [87]:
X = np.array(X_pred.drop(columns=['laptop_ID'])) # La x es todo menos el nombre
y = np.array(X_pred['laptop_ID']) # La y es el nombre

In [88]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()

regressor.fit(X, y) # Entrenar

LinearRegression()

IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [91]:
X_pred = X_pred[['Inches']]
X_pred

,Inches
0,15.6
1,14.0
2,13.3
3,15.4
4,15.6
...,...
386,13.3
387,13.3
388,15.6
389,14.0


### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle.

**¿Qué es lo que subirás a Kaggle?**

In [92]:
predictions_submit = regressor.predict(X_pred)
predictions_submit

c:\Users\nuria\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


array([645.74605285, 691.84356347, 712.01122436, 651.50824168,
       645.74605285, 645.74605285, 596.76744782, 645.74605285,
       645.74605285, 691.84356347, 645.74605285, 691.84356347,
       645.74605285, 645.74605285, 596.76744782, 596.76744782,
       760.98982939, 645.74605285, 645.74605285, 645.74605285,
       596.76744782, 712.01122436, 596.76744782, 645.74605285,
       712.01122436, 645.74605285, 645.74605285, 645.74605285,
       691.84356347, 596.76744782, 694.72465788, 712.01122436,
       645.74605285, 712.01122436, 645.74605285, 691.84356347,
       596.76744782, 596.76744782, 645.74605285, 691.84356347,
       645.74605285, 691.84356347, 706.24903553, 645.74605285,
       645.74605285, 645.74605285, 645.74605285, 596.76744782,
       645.74605285, 691.84356347, 645.74605285, 645.74605285,
       645.74605285, 645.74605285, 645.74605285, 691.84356347,
       596.76744782, 596.76744782, 596.76744782, 691.84356347,
       596.76744782, 645.74605285, 645.74605285, 645.74

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [93]:
sample = pd.read_csv(r'C:\Users\nuria\OneDrive\Escritorio\ML_laptops\data\raw_data\sample_submission.csv')
sample.head()

,laptop_ID,Price_euros
0,539,650
1,327,650
2,563,650
3,13,650
4,935,500


In [94]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [96]:
submission = pd.DataFrame({"laptop_ID": sample['laptop_ID'], "Price_euros": predictions_submit})
submission.head()

,laptop_ID,Price_euros
0,539,645.746053
1,327,691.843563
2,563,712.011224
3,13,651.508242
4,935,645.746053


### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [97]:
import urllib.request

In [98]:
def chequeator(df_to_submit,version):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit['laptop_ID'].all() == sample['laptop_ID'].all():
                print("You're ready to submit!")
                submission.to_csv(f"submission_{version}.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [99]:
chequeator(submission,1)

You're ready to submit!


URLError: <urlopen error [Errno 11001] getaddrinfo failed>